In [15]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import when

spark = SparkSession.builder \
    .appName('pyspark-run-with-gcp-bucket') \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

# Set GCS credentials if necessary
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", "D:/de_projects/de-zoomcamp/01-docker-terraform/2_docker_sql/spark/de-zoomcamp-411813-1aeafde99348.json")


# Define GCS bucket and file path
bucket_name = "mage_zoom_zhan"
file_name = "netflix_data.csv"
file_path = f"gs://{bucket_name}/{file_name}"

#split_column = "Plan_Duration"
#split_by = " "
#new_columns = ["month number", "month"]

# Read CSV file from GCS into DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)
df = df.withColumn("Gender", when(df["Gender"] == "Female", "F").otherwise(when(df["Gender"] == "Male", "M").otherwise(df["Gender"])))

# Replace 'Female' with 'F' and 'Male' with 'M' in the Gender column
#df['Gender'].replace('Female', '1', inplace=True)

# Coalesce the DataFrame to a single partition
df = df.coalesce(1)
print(df)
# Write the DataFrame to a single CSV file in GCS
df.write.csv(f"gs://{bucket_name}/netflix_out", header=True)

# Stop SparkSession
spark.stop()

+---+-------+-----------------+---------------+---------+-----------------+--------------+---+------+----------+-------------+
|_c0|User ID|Subscription Type|Monthly Revenue|Join Date|Last Payment Date|       Country|Age|Gender|    Device|Plan Duration|
+---+-------+-----------------+---------------+---------+-----------------+--------------+---+------+----------+-------------+
|  0|      1|            Basic|             10| 15-01-22|         10-06-23| United States| 28|     M|Smartphone|      1 Month|
|  1|      2|          Premium|             15| 05-09-21|         22-06-23|        Canada| 35|     F|    Tablet|      1 Month|
|  2|      3|         Standard|             12| 28-02-23|         27-06-23|United Kingdom| 42|     M|  Smart TV|      1 Month|
|  3|      4|         Standard|             12| 10-07-22|         26-06-23|     Australia| 51|     F|    Laptop|      1 Month|
|  4|      5|            Basic|             10| 01-05-23|         28-06-23|       Germany| 33|     M|Smartphone

AnalysisException: [PATH_ALREADY_EXISTS] Path gs://mage_zoom_zhan/netflix_out already exists. Set mode as "overwrite" to overwrite the existing path.